# Estudio Vid
## Pablo Lavín
## TFM - Master Ciencia de Datos (UC-UIMP)
## Beca JAE Intro ICU 2025
## Septiembre 2025 - Junio 2026

En este notebook se analiza el desempeño del modelo ECMWF frente a ERA5-Land durante el periodo 1981–2022.

Con el objetivo de mejorar la representatividad del modelo respecto a los datos observacionales, se realiza una corrección del sesgo de la temperatura máxima y mínima del modelo ECMWF.

Posteriormente los datos de temperatura se convierten en índices, en este caso, un índice de temperatura máxima (número de días que se supera una temperatura umbral) y otro de temperatura mínima, con el objetivo de estudiar las temperaturas extremas durante los meses de mayo y junio (fases críticas de floración de la vid).

Se incluyen métricas de sesgo, correlación, RMSE, ratio de varianzas y ROCSS.

## Configuración

In [1]:
# Cargamos paquetes
source("../../scripts/setup_libraries.R")
source("../../scripts/load_bc_functions.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

Loading required package: udunits2

udunits system database read from /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/share/udunits/udunits2.xml

convertR version 0.3.0 (2025-07-31) is loaded


Development version may have an unexpected behaviour

  More information about the 'climate4R' ecosystem in: http://meteo.unican.es/climate4R


Attaching package: ‘convertR’


The following objects are masked from ‘package:loadeR’:

    hurs2huss, huss2hurs, tdps2hurs


visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0

In [2]:
# Region de estudio

lon = c(-10, 5)
lat = c(35, 44)

# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

In [3]:
# Cargo datos
tmx_modelo_0 = readRDS("../../data/vine_data/tasmax_model_vid_0.rds")
tmx_modelo_1 = readRDS("../../data/vine_data/tasmax_model_vid_1.rds")
tmx_modelo_2 = readRDS("../../data/vine_data/tasmax_model_vid_2.rds")
tmx_modelo_3 = readRDS("../../data/vine_data/tasmax_model_vid_3.rds")

tmx_obs_cel = readRDS("../../data/vine_data/tasmax_obs_vid.rds")

tmn_modelo_0 = readRDS("../../data/vine_data/tasmin_model_vid_0.rds")
tmn_modelo_1 = readRDS("../../data/vine_data/tasmin_model_vid_1.rds")
tmn_modelo_2 = readRDS("../../data/vine_data/tasmin_model_vid_2.rds")
tmn_modelo_3 = readRDS("../../data/vine_data/tasmin_model_vid_3.rds")

tmn_obs_cel = readRDS("../../data/vine_data/tasmin_obs_vid.rds")

## biasCorrection (bC)

In [ ]:
folds_list = lapply(1981:2022, function(x) x)

bc_tasmax_0 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_0,
    newdata = tmx_modelo_0,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmax_1 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_1,
    newdata = tmx_modelo_1,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmax_2 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_2,
    newdata = tmx_modelo_2,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmax_3 = biasCorrection_RM(
    y = tmx_obs_cel,
    x = tmx_modelo_3,
    newdata = tmx_modelo_3,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

In [ ]:
saveRDS(bc_tasmax_0, file = "bc_tasmax_vid_0.rds")
saveRDS(bc_tasmax_1, file = "bc_tasmax_vid_1.rds")
saveRDS(bc_tasmax_2, file = "bc_tasmax_vid_2.rds")
saveRDS(bc_tasmax_3, file = "bc_tasmax_vid_3.rds")

In [ ]:
bc_tasmin_0 = biasCorrection_RM(
    y = tmn_obs_cel,
    x = tmn_modelo_0,
    newdata = tmn_modelo_0,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmin_1 = biasCorrection_RM(
    y = tmn_obs_cel,
    x = tmn_modelo_1,
    newdata = tmn_modelo_1,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmin_2 = biasCorrection_RM(
    y = tmn_obs_cel,
    x = tmn_modelo_2,
    newdata = tmn_modelo_2,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

bc_tasmin_3 = biasCorrection_RM(
    y = tmn_obs_cel,
    x = tmn_modelo_3,
    newdata = tmn_modelo_3,
    method = "eqm",
    extrapolation = "constant",
    cross.val = 'kfold',
    folds = folds_list) %>% suppressMessages %>% suppressWarnings

In [ ]:
saveRDS(bc_tasmin_0, file = "bc_tasmin_vid_0.rds")
saveRDS(bc_tasmin_1, file = "bc_tasmin_vid_1.rds")
saveRDS(bc_tasmin_2, file = "bc_tasmin_vid_2.rds")
saveRDS(bc_tasmin_3, file = "bc_tasmin_vid_3.rds")

In [4]:
# Cargo dato de BC
bc_tasmax_0 = readRDS("../../data/vine_data/bc_tasmax_vid_0.rds")
bc_tasmax_1 = readRDS("../../data/vine_data/bc_tasmax_vid_1.rds")
bc_tasmax_2 = readRDS("../../data/vine_data/bc_tasmax_vid_2.rds")
bc_tasmax_3 = readRDS("../../data/vine_data/bc_tasmax_vid_3.rds")

bc_tasmin_0 = readRDS("../../data/vine_data/bc_tasmin_vid_0.rds")
bc_tasmin_1 = readRDS("../../data/vine_data/bc_tasmin_vid_1.rds")
bc_tasmin_2 = readRDS("../../data/vine_data/bc_tasmin_vid_2.rds")
bc_tasmin_3 = readRDS("../../data/vine_data/bc_tasmin_vid_3.rds")

### Representación del sesgo de la temperatura máxima

In [8]:
# Climatología de las obs
ref = climatology(tmx_obs_cel) %>% suppressMessages %>% suppressWarnings

# Climatología del modelos con los 4 leadtime (datos con sesgo)
diff_0 = climatology(tmx_modelo_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_1 = climatology(tmx_modelo_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_2 = climatology(tmx_modelo_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_3 = climatology(tmx_modelo_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Sesgo para cada leadtime
bias_0 = gridArithmetics(diff_0, ref, operator = "-")
bias_1 = gridArithmetics(diff_1, ref, operator = "-")
bias_2 = gridArithmetics(diff_2, ref, operator = "-")
bias_3 = gridArithmetics(diff_3, ref, operator = "-")

# Representación
b_0 = spatialPlot(bias_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(-5, 5, 0.1))
b_1 = spatialPlot(bias_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(-5, 5, 0.1))
b_2 = spatialPlot(bias_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(-5, 5, 0.1))
b_3 = spatialPlot(bias_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(-5, 5, 0.1))

# Climatología de cada leadtime (datos sin sesgo)
diff0 = climatology(bc_tasmax_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff1 = climatology(bc_tasmax_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff2 = climatology(bc_tasmax_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff3 = climatology(bc_tasmax_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Cálculo del sesgo
bias0 = gridArithmetics(diff0, ref, operator = "-")
bias1 = gridArithmetics(diff1, ref, operator = "-")
bias2 = gridArithmetics(diff2, ref, operator = "-")
bias3 = gridArithmetics(diff3, ref, operator = "-")

# Representación
b0 = spatialPlot(bias0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b1 = spatialPlot(bias1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b2 = spatialPlot(bias2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b3 = spatialPlot(bias3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))

In [9]:
png("bias_tmax_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(b_0, b_1, b_2, b_3, b0, b1, b2, b3, ncol = 4)
dev.off()

pdf 
  2

### Representación del sesgo de la temperatura mínima

In [13]:
# Climatología de las obs
ref = climatology(tmn_obs_cel) %>% suppressMessages %>% suppressWarnings

# Climatología del modelos con los 4 leadtime (datos con sesgo)
diff_0 = climatology(tmn_modelo_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_1 = climatology(tmn_modelo_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_2 = climatology(tmn_modelo_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff_3 = climatology(tmn_modelo_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Sesgo para cada leadtime
bias_0 = gridArithmetics(diff_0, ref, operator = "-")
bias_1 = gridArithmetics(diff_1, ref, operator = "-")
bias_2 = gridArithmetics(diff_2, ref, operator = "-")
bias_3 = gridArithmetics(diff_3, ref, operator = "-")

# Representación
b_0 = spatialPlot(bias_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(-4, 4, 0.1))
b_1 = spatialPlot(bias_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(-4, 4, 0.1))
b_2 = spatialPlot(bias_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(-4, 4, 0.1))
b_3 = spatialPlot(bias_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(-4, 4, 0.1))

# Climatología de cada leadtime (datos sin sesgo)
diff0 = climatology(bc_tasmin_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff1 = climatology(bc_tasmin_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff2 = climatology(bc_tasmin_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
diff3 = climatology(bc_tasmin_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Cálculo del sesgo
bias0 = gridArithmetics(diff0, ref, operator = "-")
bias1 = gridArithmetics(diff1, ref, operator = "-")
bias2 = gridArithmetics(diff2, ref, operator = "-")
bias3 = gridArithmetics(diff3, ref, operator = "-")

# Representación
b0 = spatialPlot(bias0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b1 = spatialPlot(bias1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b2 = spatialPlot(bias2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))
b3 = spatialPlot(bias3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(-0.006, 0.006, 0.0001))

In [14]:
png("bias_tmin_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(b_0, b_1, b_2, b_3, b0, b1, b2, b3, ncol = 4)
dev.off()

pdf 
  2

## Máscara para los datos

In [5]:
## Calculo el número de días que tmax > 22 grados (solo para la estructura del grid)
nd_obs = indexGrid(tx = tmx_obs_cel, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

## Máscara de tierra de ERA5 (es una variable más del propio reanális):
## Valores continuos entre 0 (no hay nada de tierra en ese gridbox) y 1 (todo el gridbox es tierra)
mask = loadGridData("/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5/lsm/lsm_era5.nc", var = "lsm") %>% suppressMessages %>% suppressWarnings

## Binarizo la máscara: Considero que todos los gridboxes con un valor por encima (debajo) de 0.5 son de tierra (mar)
mask.bin = binaryGrid(mask, condition = "GE", threshold = 0.5, values = c(NA, 1))

## Hago el upscaling como hice con los datos de ERA5 a la resolución de 1º del modelo
mask_upscaled = interpGrid(mask.bin,
                           new.coordinates = getGrid(tmx_obs_cel),
                           method = "bilinear") %>% suppressMessages %>% suppressWarnings

## Apoyándome en la máscara binaria, me quedo únicamente con los datos en tierra y descarto el mar
mask.bin.spain = subsetGrid(mask_upscaled, lonLim = c(-10, 5), latLim = c(35, 44))
mask.bin.spain$Data = aperm(replicate(getShape(nd_obs)["time"], mask.bin.spain$Data, simplify = "array"), c(3, 1, 2))
attributes(mask.bin.spain$Data)$dimensions = c("time", "lat", "lon")

## Máscara para el moodelo
n.members = getShape(tmx_modelo_0)["member"]
mask.data = mask.bin.spain$Data
mask.4d = array(NA, dim = c(n.members, dim(mask.data)))  # member x time x lat x lon
for (m in 1:n.members) {
    mask.4d[m,,,] = mask.data
}

mask.model = mask.bin.spain  # copia de la estructura
mask.model$Data = mask.4d
attributes(mask.model$Data)$dimensions = c("member", "time", "lat", "lon")

## ndays tmax > 22

### obs

In [ ]:
# Calculo el número de días que tmax > 22 grados
nd_tasmax_obs = indexGrid(tx = tmx_obs_cel, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos de ERA-5
ndtasmax_obs = gridArithmetics(nd_tasmax_obs, mask.bin.spain, operator = "*")

# Computo la climatología
nd_tmax_obs = climatology(ndtasmax_obs) %>% suppressMessages %>% suppressWarnings

# Representación
ndtmaxobs = spatialPlot(nd_tmax_obs, backdrop.theme = "countries", main = "Obs", col.regions = color, at = seq(0, 61, 1))

In [ ]:
png("ndays_tmax_obs_vid.png", width = 2000, height = 1000, res = 150)
print(ndtmaxobs)
dev.off()

### datos modelo raw

In [5]:
# Calculo el número de días que tmax > 22 grados
nd_tasmax_0 = indexGrid(tx = tmx_modelo_0, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_1 = indexGrid(tx = tmx_modelo_1, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_2 = indexGrid(tx = tmx_modelo_2, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_3 = indexGrid(tx = tmx_modelo_3, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
ndtasmax_0 = gridArithmetics(nd_tasmax_0, mask.model, operator = "*")
ndtasmax_1 = gridArithmetics(nd_tasmax_1, mask.model, operator = "*")
ndtasmax_2 = gridArithmetics(nd_tasmax_2, mask.model, operator = "*")
ndtasmax_3 = gridArithmetics(nd_tasmax_3, mask.model, operator = "*")

# Computo la climatología
nd_tmax_0 = climatology(ndtasmax_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_1 = climatology(ndtasmax_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_2 = climatology(ndtasmax_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_3 = climatology(ndtasmax_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
ndtmax0 = spatialPlot(nd_tmax_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(0, 61, 1))
ndtmax1 = spatialPlot(nd_tmax_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(0, 61, 1))
ndtmax2 = spatialPlot(nd_tmax_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(0, 61, 1))
ndtmax3 = spatialPlot(nd_tmax_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(0, 61, 1))

### datos modelo bC

In [7]:
# Calculo el número de días que tmax > 22 grados
nd_tasmax_bc_0 = indexGrid(tx = bc_tasmax_0, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_bc_1 = indexGrid(tx = bc_tasmax_1, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_bc_2 = indexGrid(tx = bc_tasmax_2, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings
nd_tasmax_bc_3 = indexGrid(tx = bc_tasmax_3, index.code = "TXth", th = 22) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
ndtasmax_bc_0 = gridArithmetics(nd_tasmax_bc_0, mask.model, operator = "*")
ndtasmax_bc_1 = gridArithmetics(nd_tasmax_bc_1, mask.model, operator = "*")
ndtasmax_bc_2 = gridArithmetics(nd_tasmax_bc_2, mask.model, operator = "*")
ndtasmax_bc_3 = gridArithmetics(nd_tasmax_bc_3, mask.model, operator = "*")

# Computo la climatología
nd_tmax_bc_0 = climatology(ndtasmax_bc_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_bc_1 = climatology(ndtasmax_bc_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_bc_2 = climatology(ndtasmax_bc_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmax_bc_3 = climatology(ndtasmax_bc_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
ndtmax0_bc = spatialPlot(nd_tmax_bc_0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(0, 61, 1))
ndtmax1_bc = spatialPlot(nd_tmax_bc_1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(0, 61, 1))
ndtmax2_bc = spatialPlot(nd_tmax_bc_2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(0, 61, 1))
ndtmax3_bc = spatialPlot(nd_tmax_bc_3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(0, 61, 1))

In [24]:
png("ndays_tmax_model_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(ndtmax0, ndtmax1, ndtmax2, ndtmax3, ndtmax0_bc, ndtmax1_bc, ndtmax2_bc, ndtmax3_bc, ncol = 4)
dev.off()

pdf 
  2

## ndays tmin < 0

### obs

In [ ]:
# Calculo el número de días que tmin < 0 grados
nd_tasmin_obs = indexGrid(tn = tmn_obs_cel, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos de ERA-5
ndtasmin_obs = gridArithmetics(nd_tasmin_obs, mask.bin.spain, operator = "*")

# Computo la climatología
nd_tmin_obs = climatology(ndtasmin_obs) %>% suppressMessages %>% suppressWarnings

# Representación
ndtasminobs = spatialPlot(nd_tmin_obs, backdrop.theme = "countries", main = "Obs", col.regions = color, at = seq(0, 4, 0.1))

In [ ]:
png("ndays_tmin_obs_vid.png", width = 2000, height = 1000, res = 150)
print(ndtasminobs)
dev.off()

### datos modelo raw

In [6]:
# Calculo el número de días que tmin < 0 grados
nd_tasmin_0 = indexGrid(tn = tmn_modelo_0, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_1 = indexGrid(tn = tmn_modelo_1, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_2 = indexGrid(tn = tmn_modelo_2, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_3 = indexGrid(tn = tmn_modelo_3, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
ndtasmin_0 = gridArithmetics(nd_tasmin_0, mask.model, operator = "*")
ndtasmin_1 = gridArithmetics(nd_tasmin_1, mask.model, operator = "*")
ndtasmin_2 = gridArithmetics(nd_tasmin_2, mask.model, operator = "*")
ndtasmin_3 = gridArithmetics(nd_tasmin_3, mask.model, operator = "*")

# Computo la climatología
nd_tmin_0 = climatology(ndtasmin_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_1 = climatology(ndtasmin_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_2 = climatology(ndtasmin_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_3 = climatology(ndtasmin_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
ndtasmin0 = spatialPlot(nd_tmin_0, backdrop.theme = "countries", main = "Raw (lt 0)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin1 = spatialPlot(nd_tmin_1, backdrop.theme = "countries", main = "Raw (lt 1)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin2 = spatialPlot(nd_tmin_2, backdrop.theme = "countries", main = "Raw (lt 2)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin3 = spatialPlot(nd_tmin_3, backdrop.theme = "countries", main = "Raw (lt 3)", col.regions = color, at = seq(0, 4, 0.1))

### datos modelo bC

In [7]:
# Calculo el número de días que tmin < 0 grados
nd_tasmin_bc_0 = indexGrid(tn = bc_tasmin_0, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_bc_1 = indexGrid(tn = bc_tasmin_1, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_bc_2 = indexGrid(tn = bc_tasmin_2, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings
nd_tasmin_bc_3 = indexGrid(tn = bc_tasmin_3, index.code = "TNth", th = 0) %>% suppressMessages %>% suppressWarnings

# Aplico la máscara a mis datos del modelo
ndtasmin_bc_0 = gridArithmetics(nd_tasmin_bc_0, mask.model, operator = "*")
ndtasmin_bc_1 = gridArithmetics(nd_tasmin_bc_1, mask.model, operator = "*")
ndtasmin_bc_2 = gridArithmetics(nd_tasmin_bc_2, mask.model, operator = "*")
ndtasmin_bc_3 = gridArithmetics(nd_tasmin_bc_3, mask.model, operator = "*")

# Computo la climatología
nd_tmin_bc_0 = climatology(ndtasmin_bc_0, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_bc_1 = climatology(ndtasmin_bc_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_bc_2 = climatology(ndtasmin_bc_2, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
nd_tmin_bc_3 = climatology(ndtasmin_bc_3, by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Representación
ndtasmin0_bc = spatialPlot(nd_tmin_bc_0, backdrop.theme = "countries", main = "BC (lt 0)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin1_bc = spatialPlot(nd_tmin_bc_1, backdrop.theme = "countries", main = "BC (lt 1)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin2_bc = spatialPlot(nd_tmin_bc_2, backdrop.theme = "countries", main = "BC (lt 2)", col.regions = color, at = seq(0, 4, 0.1))
ndtasmin3_bc = spatialPlot(nd_tmin_bc_3, backdrop.theme = "countries", main = "BC (lt 3)", col.regions = color, at = seq(0, 4, 0.1))

In [37]:
png("ndays_tmin_model_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(ndtasmin0, ndtasmin1, ndtasmin2, ndtasmin3, ndtasmin0_bc, ndtasmin1_bc, ndtasmin2_bc, ndtasmin3_bc, ncol = 4)
dev.off()

pdf 
  2

## Bias ndays tmax

### obs y raw

In [39]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_tasmax_days_0 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmax_0$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_1 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmax_1$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_2 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmax_2$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_3 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmax_3$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_ndtasmax_grid_0 = easyVeri2grid(easyVeri.mat = bias_tasmax_days_0, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_1 = easyVeri2grid(easyVeri.mat = bias_tasmax_days_1, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_2 = easyVeri2grid(easyVeri.mat = bias_tasmax_days_2, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_3 = easyVeri2grid(easyVeri.mat = bias_tasmax_days_3, obs.grid = tmx_obs_cel, verifun = "EnsMe")

In [44]:
# Representación
bias_tasmax_0 = spatialPlot(climatology(bias_ndtasmax_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 0)",
                            at = seq(-20, 20, 1)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_1 = spatialPlot(climatology(bias_ndtasmax_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 1)",
                            at = seq(-20, 20, 1)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_2 = spatialPlot(climatology(bias_ndtasmax_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 2)",
                            at = seq(-20, 20, 1)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_3 = spatialPlot(climatology(bias_ndtasmax_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 3)",
                            at = seq(-20, 20, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [45]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_tasmax_days_0_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmax_bc_0$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_1_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmax_bc_1$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_2_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmax_bc_2$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmax_days_3_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmax_bc_3$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_ndtasmax_grid_0_bc = easyVeri2grid(easyVeri.mat = bias_tasmax_days_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_1_bc = easyVeri2grid(easyVeri.mat = bias_tasmax_days_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_2_bc = easyVeri2grid(easyVeri.mat = bias_tasmax_days_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")
bias_ndtasmax_grid_3_bc = easyVeri2grid(easyVeri.mat = bias_tasmax_days_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsMe")

In [50]:
# Representación
bias_tasmax_0_bc = spatialPlot(climatology(bias_ndtasmax_grid_0_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 0)",
                               at = seq(-0.08, 0.08, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_1_bc = spatialPlot(climatology(bias_ndtasmax_grid_1_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 1)",
                               at = seq(-0.08, 0.08, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_2_bc = spatialPlot(climatology(bias_ndtasmax_grid_2_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 2)",
                               at = seq(-0.08, 0.08, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmax_3_bc = spatialPlot(climatology(bias_ndtasmax_grid_3_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 3)",
                               at = seq(-0.08, 0.08, 0.001)) %>% suppressMessages %>% suppressWarnings

In [51]:
png("bias_ndays_tmax_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(bias_tasmax_0, bias_tasmax_1, bias_tasmax_2, bias_tasmax_3,
             bias_tasmax_0_bc, bias_tasmax_1_bc, bias_tasmax_2_bc, bias_tasmax_3_bc, ncol = 4)
dev.off()

pdf 
  2

## Bias ndays tmin

### obs y raw

In [69]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_tasmin_days_0 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmin_0$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_1 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmin_1$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_2 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmin_2$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_3 = veriApply(verifun = "EnsMe", 
                               fcst = ndtasmin_3$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_ndtasmin_grid_0 = easyVeri2grid(easyVeri.mat = bias_tasmin_days_0, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_1 = easyVeri2grid(easyVeri.mat = bias_tasmin_days_1, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_2 = easyVeri2grid(easyVeri.mat = bias_tasmin_days_2, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_3 = easyVeri2grid(easyVeri.mat = bias_tasmin_days_3, obs.grid = tmn_obs_cel, verifun = "EnsMe")

In [70]:
# Representación
bias_tasmin_0 = spatialPlot(climatology(bias_ndtasmin_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 0)",
                            at = seq(-0.6, 0.6, 0.1)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_1 = spatialPlot(climatology(bias_ndtasmin_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 1)",
                            at = seq(-0.6, 0.6, 0.1)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_2 = spatialPlot(climatology(bias_ndtasmin_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 2)",
                            at = seq(-0.6, 0.6, 0.1)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_3 = spatialPlot(climatology(bias_ndtasmin_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 3)",
                            at = seq(-0.6, 0.6, 0.1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [71]:
# Calculo del bias del ndays para cada leadtime del modelo
bias_tasmin_days_0_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmin_bc_0$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_1_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmin_bc_1$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_2_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmin_bc_2$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

bias_tasmin_days_3_bc = veriApply(verifun = "EnsMe", 
                                  fcst = ndtasmin_bc_3$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_ndtasmin_grid_0_bc = easyVeri2grid(easyVeri.mat = bias_tasmin_days_0_bc, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_1_bc = easyVeri2grid(easyVeri.mat = bias_tasmin_days_1_bc, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_2_bc = easyVeri2grid(easyVeri.mat = bias_tasmin_days_2_bc, obs.grid = tmn_obs_cel, verifun = "EnsMe")
bias_ndtasmin_grid_3_bc = easyVeri2grid(easyVeri.mat = bias_tasmin_days_3_bc, obs.grid = tmn_obs_cel, verifun = "EnsMe")

In [76]:
# Representación
bias_tasmin_0_bc = spatialPlot(climatology(bias_ndtasmin_grid_0_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 0)",
                               at = seq(-0.05, 0.05, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_1_bc = spatialPlot(climatology(bias_ndtasmin_grid_1_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 1)",
                               at = seq(-0.05, 0.05, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_2_bc = spatialPlot(climatology(bias_ndtasmin_grid_2_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 2)",
                               at = seq(-0.05, 0.05, 0.001)) %>% suppressMessages %>% suppressWarnings

bias_tasmin_3_bc = spatialPlot(climatology(bias_ndtasmin_grid_3_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 3)",
                               at = seq(-0.05, 0.05, 0.001)) %>% suppressMessages %>% suppressWarnings

In [77]:
png("bias_ndays_tmin_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(bias_tasmin_0, bias_tasmin_1, bias_tasmin_2, bias_tasmin_3,
             bias_tasmin_0_bc, bias_tasmin_1_bc, bias_tasmin_2_bc, bias_tasmin_3_bc, ncol = 4)
dev.off()

pdf 
  2

## Correlación ndays tmax

### test de hipótesis para puntos significativos

In [10]:
# Función para calcular correlación de Pearson y valores p entre datos de modelo y observaciones en una grilla espacial
# Además, identifica y marca los puntos con correlación estadísticamente significativa según un umbral de p-valor
#
# Args:
#   model_data: objeto con datos del modelo, estructura esperada con dimensión [miembros, tiempo, latitud, longitud]
#   obs_data: objeto con datos observacionales, estructura con dimensión [tiempo, latitud, longitud]
#   ref_grid: objeto referencia con metadatos espaciales y temporales para construir grillas (xyCoords, Variable, Dates)
#   threshold: umbral para marcar significancia estadística (p-valor), default 0.05
#
# Returns:
#   Lista con:
#     - cor: matriz de correlaciones [lat x lon]
#     - pval: matriz de valores p [lat x lon]
#     - pval_grid: objeto tipo "grid" con valores p y metadatos
#     - pts: lista de objetos para graficar puntos de significancia (stippling)

calc_cor_pval_grid = function(model_data, obs_data, ref_grid, threshold = 0.05) {
    
    # Calcular la media del ensamble para cada punto [tiempo, lat, lon]
    ens_mean = apply(model_data$Data, c(2, 3, 4), mean, na.rm = TRUE)
    
    # Dimensiones espaciales (latitud y longitud)
    lat_n = dim(ens_mean)[2]
    lon_n = dim(ens_mean)[3]
  
    # Inicializar matrices vacías para almacenar correlaciones y p-valores
    cor_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    pval_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    
    # Iterar sobre cada punto espacial
    for (i in 1:lat_n) {
        for (j in 1:lon_n) {
            
            # Extraer series temporales de modelo y observaciones para la celda actual
            pred_series = ens_mean[, i, j]
            obs_series = obs_data$Data[, i, j]
      
            # Filtrar índices con datos completos (no NA)
            valid_idx = complete.cases(pred_series, obs_series)
            
            # Solo calcular correlación si hay suficientes datos (mínimo 10)
            if (sum(valid_idx) >= 10) {
                test = cor.test(pred_series[valid_idx], obs_series[valid_idx], method = "pearson")
                cor_array[i, j] = test$estimate  # Coeficiente de correlación
                pval_array[i, j] = test$p.value  # Valor p de la prueba
            }
        }
    }
  
    # Construir un objeto "grid" para los valores p, con metadatos espaciales y temporales
    pval_grid = list()
    pval_grid$Data = pval_array
    attr(pval_grid$Data, "dimensions") = c("lat", "lon")
    pval_grid$xyCoords = ref_grid$xyCoords
    pval_grid$Variable = ref_grid$Variable
    pval_grid$Dates = ref_grid$Dates
    class(pval_grid) = "grid"

    pval_grid$Variable$varName = "p-values"
    attr(pval_grid$Variable, "description") = "Mapa de p-valores"
    attr(pval_grid$Variable, "units") = ""
    attr(pval_grid$Variable, "longname") = "p-values"
    
    # Crear objetos para graficar puntos de significancia estadística (stippling)
    pts = map.stippling(climatology(pval_grid), 
                        threshold = threshold, 
                        condition = "LT", 
                        pch = 19, col = "black", cex = 0.5) %>% suppressMessages() %>% suppressWarnings()
    
    # Devolver lista con resultados y objetos para plot
    return(list(cor = cor_array, pval = pval_array, pval_grid = pval_grid, pts = pts))
}

In [11]:
# Datos raw tmax
test_cor_tmax_0 = calc_cor_pval_grid(ndtasmax_0, ndtasmax_obs, ndtasmax_0)
test_cor_tmax_1 = calc_cor_pval_grid(ndtasmax_1, ndtasmax_obs, ndtasmax_1)
test_cor_tmax_2 = calc_cor_pval_grid(ndtasmax_2, ndtasmax_obs, ndtasmax_2)
test_cor_tmax_3 = calc_cor_pval_grid(ndtasmax_3, ndtasmax_obs, ndtasmax_3)

# Datos bC tmax
test_cor_tmax_0_bc = calc_cor_pval_grid(ndtasmax_bc_0, ndtasmax_obs, ndtasmax_bc_0)
test_cor_tmax_1_bc = calc_cor_pval_grid(ndtasmax_bc_1, ndtasmax_obs, ndtasmax_bc_1)
test_cor_tmax_2_bc = calc_cor_pval_grid(ndtasmax_bc_2, ndtasmax_obs, ndtasmax_bc_2)
test_cor_tmax_3_bc = calc_cor_pval_grid(ndtasmax_bc_3, ndtasmax_obs, ndtasmax_bc_3)

### obs y raw

In [12]:
# Calculo de correlacion para cada leadtime del modelo
cor_ndays_tasmax_0 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmax_0$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_1 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmax_1$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_2 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmax_2$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_3 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmax_3$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_ndtasmax_grid_0 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_0, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_1 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_1, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_2 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_2, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_3 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_3, obs.grid = tmx_obs_cel, verifun = "EnsCorr")

In [13]:
# Representación
c0_tmax = spatialPlot(climatology(cor_ndtasmax_grid_0),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmax_0$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_tmax = spatialPlot(climatology(cor_ndtasmax_grid_1),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmax_1$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_tmax = spatialPlot(climatology(cor_ndtasmax_grid_2),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmax_2$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_tmax = spatialPlot(climatology(cor_ndtasmax_grid_3),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmax_3$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

### obs y bC

In [14]:
# Calculo de correlacion para cada leadtime del modelo
cor_ndays_tasmax_0_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmax_bc_0$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_1_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmax_bc_1$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_2_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmax_bc_2$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmax_3_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmax_bc_3$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_ndtasmax_grid_0_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_1_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_2_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")
cor_ndtasmax_grid_3_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmax_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsCorr")

In [15]:
# Representación
c0_tmax_bc = spatialPlot(climatology(cor_ndtasmax_grid_0_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmax_0_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_tmax_bc = spatialPlot(climatology(cor_ndtasmax_grid_1_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmax_1_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_tmax_bc = spatialPlot(climatology(cor_ndtasmax_grid_2_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmax_2_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_tmax_bc = spatialPlot(climatology(cor_ndtasmax_grid_3_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmax_3_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [16]:
png("cor_ndays_tmax_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(c0_tmax, c1_tmax, c2_tmax, c3_tmax, c0_tmax_bc, c1_tmax_bc, c2_tmax_bc, c3_tmax_bc, ncol = 4)
dev.off()

pdf 
  2

## Correlación ndays tmin

### test de hipótesis para puntos significativos

In [17]:
# Datos raw tmin
test_cor_tmin_0 = calc_cor_pval_grid(ndtasmin_0, ndtasmin_obs, ndtasmin_0)
test_cor_tmin_1 = calc_cor_pval_grid(ndtasmin_1, ndtasmin_obs, ndtasmin_1)
test_cor_tmin_2 = calc_cor_pval_grid(ndtasmin_2, ndtasmin_obs, ndtasmin_2)
test_cor_tmin_3 = calc_cor_pval_grid(ndtasmin_3, ndtasmin_obs, ndtasmin_3)

# Datos bC tmin
test_cor_tmin_0_bc = calc_cor_pval_grid(ndtasmin_bc_0, ndtasmin_obs, ndtasmin_bc_0)
test_cor_tmin_1_bc = calc_cor_pval_grid(ndtasmin_bc_1, ndtasmin_obs, ndtasmin_bc_1)
test_cor_tmin_2_bc = calc_cor_pval_grid(ndtasmin_bc_2, ndtasmin_obs, ndtasmin_bc_2)
test_cor_tmin_3_bc = calc_cor_pval_grid(ndtasmin_bc_3, ndtasmin_obs, ndtasmin_bc_3)

Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation is zero”
Warning message in cor(x, y):
“the standard deviation i

### obs y raw

In [18]:
# Cálculo de correlación para cada leadtime del modelo
cor_ndays_tasmin_0 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmin_0$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_1 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmin_1$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_2 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmin_2$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_3 = veriApply(verifun = "EnsCorr",
                               fcst = ndtasmin_3$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
cor_ndtasmin_grid_0 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_0, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_1 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_1, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_2 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_2, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_3 = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_3, obs.grid = tmn_obs_cel, verifun = "EnsCorr")

In [19]:
# Representación
c0_tmin = spatialPlot(climatology(cor_ndtasmin_grid_0),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmin_0$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_tmin = spatialPlot(climatology(cor_ndtasmin_grid_1),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmin_1$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_tmin = spatialPlot(climatology(cor_ndtasmin_grid_2),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmin_2$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_tmin = spatialPlot(climatology(cor_ndtasmin_grid_3),
                      backdrop.theme = "countries",
                      sp.layout = list(test_cor_tmin_3$pts),
                      col.regions = color,
                      at = seq(-1, 1, 0.1),
                      main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

### obs y bC

In [20]:
# Cálculo de correlación para cada leadtime del modelo
cor_ndays_tasmin_0_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmin_bc_0$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_1_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmin_bc_1$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_2_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmin_bc_2$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

cor_ndays_tasmin_3_bc = veriApply(verifun = "EnsCorr",
                                  fcst = ndtasmin_bc_3$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid (tmin bc)
cor_ndtasmin_grid_0_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_0_bc, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_1_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_1_bc, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_2_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_2_bc, obs.grid = tmn_obs_cel, verifun = "EnsCorr")
cor_ndtasmin_grid_3_bc = easyVeri2grid(easyVeri.mat = cor_ndays_tasmin_3_bc, obs.grid = tmn_obs_cel, verifun = "EnsCorr")

In [21]:
# Representación
c0_tmin_bc = spatialPlot(climatology(cor_ndtasmin_grid_0_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmin_0_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

c1_tmin_bc = spatialPlot(climatology(cor_ndtasmin_grid_1_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmin_1_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

c2_tmin_bc = spatialPlot(climatology(cor_ndtasmin_grid_2_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmin_2_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

c3_tmin_bc = spatialPlot(climatology(cor_ndtasmin_grid_3_bc),
                         backdrop.theme = "countries",
                         sp.layout = list(test_cor_tmin_3_bc$pts),
                         col.regions = color,
                         at = seq(-1, 1, 0.1),
                         main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [22]:
png("cor_ndays_tmin_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(c0_tmin, c1_tmin, c2_tmin, c3_tmin, c0_tmin_bc, c1_tmin_bc, c2_tmin_bc, c3_tmin_bc, ncol = 4)
dev.off()

pdf 
  2

## RMSE ndays tmax

### obs y raw

In [23]:
# Calculo del rmse del ndays para cada leadtime del modelo
rmse_tasmax_days_0 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmax_0$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_1 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmax_1$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_2 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmax_2$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_3 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmax_3$Data, 
                               obs = ndtasmax_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_ndtasmax_grid_0 = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_0, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_1 = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_1, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_2 = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_2, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_3 = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_3, obs.grid = tmx_obs_cel, verifun = "EnsRmse")

In [43]:
# Representación
rmse_tasmax_0 = spatialPlot(climatology(rmse_ndtasmax_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 0)",
                            at = seq(0, 16, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_1 = spatialPlot(climatology(rmse_ndtasmax_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 1)",
                            at = seq(0, 16, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_2 = spatialPlot(climatology(rmse_ndtasmax_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 2)",
                            at = seq(0, 16, 1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_3 = spatialPlot(climatology(rmse_ndtasmax_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 3)",
                            at = seq(0, 16, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [29]:
# Calculo del rmse del ndays para cada leadtime del modelo
rmse_tasmax_days_0_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmax_bc_0$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_1_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmax_bc_1$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_2_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmax_bc_2$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_days_3_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmax_bc_3$Data, 
                                  obs = ndtasmax_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_ndtasmax_grid_0_bc = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_0_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_1_bc = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_1_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_2_bc = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_2_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")
rmse_ndtasmax_grid_3_bc = easyVeri2grid(easyVeri.mat = rmse_tasmax_days_3_bc, obs.grid = tmx_obs_cel, verifun = "EnsRmse")

In [44]:
# Representación
rmse_tasmax_0_bc = spatialPlot(climatology(rmse_ndtasmax_grid_0_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 0)",
                               at = seq(0, 16, 0.5)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_1_bc = spatialPlot(climatology(rmse_ndtasmax_grid_1_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 1)",
                               at = seq(0, 16, 0.5)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_2_bc = spatialPlot(climatology(rmse_ndtasmax_grid_2_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 2)",
                               at = seq(0, 16, 0.5)) %>% suppressMessages %>% suppressWarnings

rmse_tasmax_3_bc = spatialPlot(climatology(rmse_ndtasmax_grid_3_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 3)",
                               at = seq(0, 16, 0.5)) %>% suppressMessages %>% suppressWarnings

In [45]:
png("rmse_ndays_tmax_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rmse_tasmax_0, rmse_tasmax_1, rmse_tasmax_2, rmse_tasmax_3,
             rmse_tasmax_0_bc, rmse_tasmax_1_bc, rmse_tasmax_2_bc, rmse_tasmax_3_bc, ncol = 4)
dev.off()

pdf 
  2

## RMSE ndays tmin

### obs y raw

In [8]:
# Calculo del rmse del ndays para cada leadtime del modelo
rmse_tasmin_days_0 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmin_0$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_1 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmin_1$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_2 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmin_2$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_3 = veriApply(verifun = "EnsRmse", 
                               fcst = ndtasmin_3$Data, 
                               obs = ndtasmin_obs$Data, 
                               ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_ndtasmin_grid_0 = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_0, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_1 = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_1, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_2 = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_2, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_3 = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_3, obs.grid = tmn_obs_cel, verifun = "EnsRmse")

In [9]:
# Representación
rmse_tasmin_0 = spatialPlot(climatology(rmse_ndtasmin_grid_0),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 0)",
                            at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_1 = spatialPlot(climatology(rmse_ndtasmin_grid_1),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 1)",
                            at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_2 = spatialPlot(climatology(rmse_ndtasmin_grid_2),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 2)",
                            at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_3 = spatialPlot(climatology(rmse_ndtasmin_grid_3),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Raw (lt 3)",
                            at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

### obs y bC

In [10]:
# Cálculo del rmse del ndays para cada leadtime del modelo
rmse_tasmin_days_0_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmin_bc_0$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_1_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmin_bc_1$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_2_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmin_bc_2$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_days_3_bc = veriApply(verifun = "EnsRmse", 
                                  fcst = ndtasmin_bc_3$Data, 
                                  obs = ndtasmin_obs$Data, 
                                  ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
rmse_ndtasmin_grid_0_bc = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_0_bc, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_1_bc = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_1_bc, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_2_bc = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_2_bc, obs.grid = tmn_obs_cel, verifun = "EnsRmse")
rmse_ndtasmin_grid_3_bc = easyVeri2grid(easyVeri.mat = rmse_tasmin_days_3_bc, obs.grid = tmn_obs_cel, verifun = "EnsRmse")

In [11]:
# Representación
rmse_tasmin_0_bc = spatialPlot(climatology(rmse_ndtasmin_grid_0_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 0)",
                               at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_1_bc = spatialPlot(climatology(rmse_ndtasmin_grid_1_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 1)",
                               at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_2_bc = spatialPlot(climatology(rmse_ndtasmin_grid_2_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 2)",
                               at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

rmse_tasmin_3_bc = spatialPlot(climatology(rmse_ndtasmin_grid_3_bc),
                               backdrop.theme = "countries",
                               col.regions = color,
                               main = "BC (lt 3)",
                               at = seq(0, 6, 0.1)) %>% suppressMessages %>% suppressWarnings

In [12]:
png("rmse_ndays_tmin_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rmse_tasmin_0, rmse_tasmin_1, rmse_tasmin_2, rmse_tasmin_3,
             rmse_tasmin_0_bc, rmse_tasmin_1_bc, rmse_tasmin_2_bc, rmse_tasmin_3_bc, ncol = 4)
dev.off()

pdf 
  2

## Ratio varianzas ndays tmax

### obs y raw

In [86]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
var_tasmax_0 = climatology(ndtasmax_0,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_0 = climatology(var_tasmax_0,
                                 clim.fun = list(FUN = "mean", na.rm = TRUE),
                                 by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_1 = climatology(ndtasmax_1,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_1 = climatology(var_tasmax_1,
                                 clim.fun = list(FUN = "mean", na.rm = TRUE),
                                 by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_2 = climatology(ndtasmax_2,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_2 = climatology(var_tasmax_2,
                                 clim.fun = list(FUN = "mean", na.rm = TRUE),
                                 by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_3 = climatology(ndtasmax_3,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_3 = climatology(var_tasmax_3,
                                 clim.fun = list(FUN = "mean", na.rm = TRUE),
                                 by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_obs = climatology(ndtasmax_obs,
                             clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_tasmax_0 = gridArithmetics(mean_var_tasmax_0, var_tasmax_obs, operator = "/")
rv_tasmax_1 = gridArithmetics(mean_var_tasmax_1, var_tasmax_obs, operator = "/")
rv_tasmax_2 = gridArithmetics(mean_var_tasmax_2, var_tasmax_obs, operator = "/")
rv_tasmax_3 = gridArithmetics(mean_var_tasmax_3, var_tasmax_obs, operator = "/")

# Representación
rv0_tasmax = spatialPlot(rv_tasmax_0, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 0)", at = seq(0, 2, 0.1))
rv1_tasmax = spatialPlot(rv_tasmax_1, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 1)", at = seq(0, 2, 0.1))
rv2_tasmax = spatialPlot(rv_tasmax_2, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 2)", at = seq(0, 2, 0.1))
rv3_tasmax = spatialPlot(rv_tasmax_3, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 3)", at = seq(0, 2, 0.1))

### obs y bC

In [87]:
# Varianzas, calculo la varianza de cada miembro y luego hago el promedio
var_tasmax_0_bc = climatology(ndtasmax_bc_0,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_0_bc = climatology(var_tasmax_0_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_1_bc = climatology(ndtasmax_bc_1,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_1_bc = climatology(var_tasmax_1_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_2_bc = climatology(ndtasmax_bc_2,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_2_bc = climatology(var_tasmax_2_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_3_bc = climatology(ndtasmax_bc_3,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmax_3_bc = climatology(var_tasmax_3_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmax_obs = climatology(ndtasmax_obs,
                             clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_tasmax_0_bc = gridArithmetics(mean_var_tasmax_0_bc, var_tasmax_obs, operator = "/")
rv_tasmax_1_bc = gridArithmetics(mean_var_tasmax_1_bc, var_tasmax_obs, operator = "/")
rv_tasmax_2_bc = gridArithmetics(mean_var_tasmax_2_bc, var_tasmax_obs, operator = "/")
rv_tasmax_3_bc = gridArithmetics(mean_var_tasmax_3_bc, var_tasmax_obs, operator = "/")

# Representación
rv0_tasmax_bc = spatialPlot(rv_tasmax_0_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 0)", at = seq(0, 2, 0.1))
rv1_tasmax_bc = spatialPlot(rv_tasmax_1_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 1)", at = seq(0, 2, 0.1))
rv2_tasmax_bc = spatialPlot(rv_tasmax_2_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 2)", at = seq(0, 2, 0.1))
rv3_tasmax_bc = spatialPlot(rv_tasmax_3_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 3)", at = seq(0, 2, 0.1))

In [88]:
png("ratiovars_ndays_tmax_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rv0_tasmax, rv1_tasmax, rv2_tasmax, rv3_tasmax,
             rv0_tasmax_bc, rv1_tasmax_bc, rv2_tasmax_bc, rv3_tasmax_bc, ncol = 4)
dev.off()

pdf 
  2

## Ratio varianzas ndays tmin

### obs y raw

In [97]:
# Varianzas, cálculo de la varianza de cada miembro y luego promedio
var_tasmin_0 = climatology(ndtasmin_0,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_0 = climatology(var_tasmin_0,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_1 = climatology(ndtasmin_1,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_1 = climatology(var_tasmin_1,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_2 = climatology(ndtasmin_2,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_2 = climatology(var_tasmin_2,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_3 = climatology(ndtasmin_3,
                           clim.fun = list(FUN = "var", na.rm = TRUE),
                           by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_3 = climatology(var_tasmin_3,
                                clim.fun = list(FUN = "mean", na.rm = TRUE),
                                by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_obs = climatology(ndtasmin_obs,
                             clim.fun = list(FUN = "var", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_tasmin_0 = gridArithmetics(mean_var_tasmin_0, var_tasmin_obs, operator = "/")
rv_tasmin_1 = gridArithmetics(mean_var_tasmin_1, var_tasmin_obs, operator = "/")
rv_tasmin_2 = gridArithmetics(mean_var_tasmin_2, var_tasmin_obs, operator = "/")
rv_tasmin_3 = gridArithmetics(mean_var_tasmin_3, var_tasmin_obs, operator = "/")

# Representación
rv0_tasmin = spatialPlot(rv_tasmin_0, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 0)", at = seq(0, 6, 0.5))
rv1_tasmin = spatialPlot(rv_tasmin_1, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 1)", at = seq(0, 6, 0.5))
rv2_tasmin = spatialPlot(rv_tasmin_2, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 2)", at = seq(0, 6, 0.5))
rv3_tasmin = spatialPlot(rv_tasmin_3, backdrop.theme = "countries", col.regions = color, main = "Raw (lt 3)", at = seq(0, 6, 0.5))

### obs y bC

In [96]:
# Varianzas, cálculo de la varianza de cada miembro y luego promedio
var_tasmin_0_bc = climatology(ndtasmin_bc_0,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_0_bc = climatology(var_tasmin_0_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_1_bc = climatology(ndtasmin_bc_1,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_1_bc = climatology(var_tasmin_1_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_2_bc = climatology(ndtasmin_bc_2,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_2_bc = climatology(var_tasmin_2_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

var_tasmin_3_bc = climatology(ndtasmin_bc_3,
                              clim.fun = list(FUN = "var", na.rm = TRUE),
                              by.member = TRUE) %>% suppressMessages %>% suppressWarnings

mean_var_tasmin_3_bc = climatology(var_tasmin_3_bc,
                                   clim.fun = list(FUN = "mean", na.rm = TRUE),
                                   by.member = FALSE) %>% suppressMessages %>% suppressWarnings

# Ratio de varianzas
rv_tasmin_0_bc = gridArithmetics(mean_var_tasmin_0_bc, var_tasmin_obs, operator = "/")
rv_tasmin_1_bc = gridArithmetics(mean_var_tasmin_1_bc, var_tasmin_obs, operator = "/")
rv_tasmin_2_bc = gridArithmetics(mean_var_tasmin_2_bc, var_tasmin_obs, operator = "/")
rv_tasmin_3_bc = gridArithmetics(mean_var_tasmin_3_bc, var_tasmin_obs, operator = "/")

# Representación
rv0_tasmin_bc = spatialPlot(rv_tasmin_0_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 0)", at = seq(0, 3, 0.1))
rv1_tasmin_bc = spatialPlot(rv_tasmin_1_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 1)", at = seq(0, 3, 0.1))
rv2_tasmin_bc = spatialPlot(rv_tasmin_2_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 2)", at = seq(0, 3, 0.1))
rv3_tasmin_bc = spatialPlot(rv_tasmin_3_bc, backdrop.theme = "countries", col.regions = color, main = "BC (lt 3)", at = seq(0, 3, 0.1))

In [98]:
png("ratiovars_ndays_tmin_vid.png", width = 2000, height = 1000, res = 150)
grid.arrange(rv0_tasmin, rv1_tasmin, rv2_tasmin, rv3_tasmin,
             rv0_tasmin_bc, rv1_tasmin_bc, rv2_tasmin_bc, rv3_tasmin_bc, ncol = 4)
dev.off()

pdf 
  2

## ROCSS ndays tmax

### Funciones auxiliares

In [151]:
## Función para realizar bootstrap y calcular ROCSS
##
## Esta función calcula el ROC Skill Score (ROCSS) mediante un enfoque bootstrap,
## generando múltiples muestras re-muestreadas con reemplazo sobre la dimensión temporal.
##
## Argumentos:
## - nd_modelo: objeto con los pronósticos originales (formato esperado: [member, time, lat, lon])
## - nd_obs: objeto con las observaciones correspondientes (formato: [time, lat, lon])
## - n_boot: número de muestras bootstrap a generar (por defecto: 1000)
##
## Valor:
## - Lista de 3 arrays (uno por tercil), cada uno con dimensiones [n_boot, lat, lon],
##   que contiene los valores bootstrap del ROCSS para cada tercil.

rocss_bootstrap = function(nd_modelo, nd_obs, n_boot = 1000) {
    
    # Obtener dimensiones del array de pronóstico
    dims = dim(nd_modelo$Data)
    
    # Inicializar lista para guardar ROCSS bootstrap para los tres terciles
    rocss_bootstrap = vector("list", length = 3)
    
    # Extraer observaciones
    obs = nd_obs$Data
    
    # Inicializar arrays vacíos para cada tercil: [n_boot, lat, lon]
    for (i in 1:3) {
        rocss_bootstrap[[i]] = array(NA, dim = c(n_boot, dims[3], dims[4]))
    }
    
    # Bucle principal de bootstrap
    for (b in 1:n_boot) {
        
        # Muestreo con reemplazo sobre la dimensión temporal (índice de tiempo)
        time_idx = sample(1:dims[2], size = dims[2], replace = TRUE)
    
        # Crear pronóstico re-muestreado: [member, time*, sdate, lat, lon]
        fcst_boot = nd_modelo$Data[, time_idx, , , drop = FALSE]
    
        # Calcular ROCSS con veriApply usando la función EnsRocss
        boot_result = veriApply(
            verifun = "EnsRocss",
            fcst = fcst_boot,
            obs = obs,
            prob = c(1/3, 2/3),
            ensdim = 1,   # Dimensión de los miembros del ensamble
            tdim = 2      # Dimensión temporal
        )
    
        # Guardar el resultado para cada tercil en la lista correspondiente
        for (i in 1:3) {
            rocss_bootstrap[[i]][b, , ] = boot_result[[i]]
        }
    }
    
    # Devolver lista con resultados bootstrap por tercil
    return(rocss_bootstrap)
}

In [152]:
## Función para generar capas de "stippling" que indican puntos con ROCSS significativamente mayores que un percentil dado
##
## Parámetros:
## mg               : objeto con datos ROCSS con dimensiones [tercil, member, time, lat, lon]
## rocss_bootstrap  : lista de arrays bootstrap con dimensiones [n_boot, lat, lon] para cada tercil
## ref_grid         : grid de referencia que contiene Dates y xyCoords (coordenadas XY)
## threshold        : percentil del bootstrap para determinar significancia (default 0.95)
##
## Retorna:
## lista de objetos "sp.points" para cada tercil con puntos significativos (stippling)

get_rocss_stippling_layers = function(mg, rocss_bootstrap, ref_grid, threshold = 0.95) {
	
	rocss_orig = mg$Data
	n_terciles = dim(rocss_orig)[1]
	n_lat = dim(rocss_orig)[4]
	n_lon = dim(rocss_orig)[5]
	
	xyCoords = ref_grid$xyCoords
	Dates = ref_grid$Dates
	
	stippling_list = vector("list", n_terciles)
	
	for (t in 1:n_terciles) {
		signif_mask = matrix(NA, nrow = n_lat, ncol = n_lon)
		
		for (i in 1:n_lat) {
			for (j in 1:n_lon) {
				boot_vals = rocss_bootstrap[[t]][, i, j]
				val_orig = rocss_orig[t, 1, 1, i, j]
				if (all(is.na(boot_vals)) || is.na(val_orig)) next
				q95 = quantile(boot_vals, threshold, na.rm = TRUE)
				signif_mask[i, j] = val_orig > q95
			}
		}
		
		# Construcción manual del objeto grid para la capa de significancia
		pval_grid = list()
		pval_grid$Data = signif_mask
		attr(pval_grid$Data, "dimensions") = c("lat", "lon")
		pval_grid$Dates = Dates
		pval_grid$xyCoords = xyCoords
		pval_grid$Variable = list(varName = paste0("significance_tercil_", t))
		class(pval_grid) = "grid"
		
		# Genera puntos de significancia con map.stippling (pch=19, color negro, tamaño 0.5)
		stippling_list[[t]] = suppressWarnings(
			suppressMessages(
				map.stippling(climatology(pval_grid), threshold = 0.5, condition = "GT",
				              pch = 19, col = "black", cex = 0.5)
			)
		)
	}
	
	# Elimina entradas NULL (si algún tercil no tiene puntos significativos)
	stippling_list = Filter(Negate(is.null), stippling_list)
	
	# Filtra para quedarse solo con listas que son puntos espaciales (sp.points)
	stippling_list = Filter(function(x) {
		is.list(x) && length(x) > 1 && x[[1]] == "sp.points"
	}, stippling_list)
	
	return(stippling_list)
}

### obs y raw

In [153]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_ndays_tasmax_0 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmax_0$Data, 
                                 obs = ndtasmax_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_1 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmax_1$Data, 
                                 obs = ndtasmax_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_2 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmax_2$Data, 
                                 obs = ndtasmax_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_3 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmax_3$Data, 
                                 obs = ndtasmax_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_tasmax_0 = makeMultiGrid(lapply(rocss_ndays_tasmax_0[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_1 = makeMultiGrid(lapply(rocss_ndays_tasmax_1[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_2 = makeMultiGrid(lapply(rocss_ndays_tasmax_2[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_3 = makeMultiGrid(lapply(rocss_ndays_tasmax_3[1:3], "easyVeri2grid", ndtasmax_obs))

In [154]:
set.seed(123)  # Reproducibilidad

# Aplico función de bootstraping
rocss_bootstrap_tasmax_0 = rocss_bootstrap(ndtasmax_0, ndtasmax_obs)
rocss_bootstrap_tasmax_1 = rocss_bootstrap(ndtasmax_1, ndtasmax_obs)
rocss_bootstrap_tasmax_2 = rocss_bootstrap(ndtasmax_2, ndtasmax_obs)
rocss_bootstrap_tasmax_3 = rocss_bootstrap(ndtasmax_3, ndtasmax_obs)

In [ ]:
saveRDS(rocss_bootstrap_tasmax_0, file = "rocss_bootstrap_tasmax_raw_oliv_0.rds")
saveRDS(rocss_bootstrap_tasmax_1, file = "rocss_bootstrap_tasmax_raw_oliv_1.rds")
saveRDS(rocss_bootstrap_tasmax_2, file = "rocss_bootstrap_tasmax_raw_oliv_2.rds")
saveRDS(rocss_bootstrap_tasmax_3, file = "rocss_bootstrap_tasmax_raw_oliv_3.rds")

In [ ]:
rocss_bootstrap_tasmax_0 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_raw_oliv_0.rds")
rocss_bootstrap_tasmax_1 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_raw_oliv_1.rds")
rocss_bootstrap_tasmax_2 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_raw_oliv_2.rds")
rocss_bootstrap_tasmax_3 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_raw_oliv_3.rds")

In [93]:
# Aplico función de máscara significativa
pts_layers_tasmax_0 = get_rocss_stippling_layers(mg_tasmax_0, rocss_bootstrap_tasmax_0, ndtasmax_0)
pts_layers_tasmax_1 = get_rocss_stippling_layers(mg_tasmax_1, rocss_bootstrap_tasmax_1, ndtasmax_0)
pts_layers_tasmax_2 = get_rocss_stippling_layers(mg_tasmax_2, rocss_bootstrap_tasmax_2, ndtasmax_0)
pts_layers_tasmax_3 = get_rocss_stippling_layers(mg_tasmax_3, rocss_bootstrap_tasmax_3, ndtasmax_0)

In [94]:
rocss_tasmax_0 = spatialPlot(climatology(mg_tasmax_0),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmax_0),
                             main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_1 = spatialPlot(climatology(mg_tasmax_1),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmax_1),
                             main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_2 = spatialPlot(climatology(mg_tasmax_2),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmax_2),
                             main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_3 = spatialPlot(climatology(mg_tasmax_3),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmax_3),
                             main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

In [95]:
png("rocss_ndays_tmax_raw_oliv.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_tasmax_0, rocss_tasmax_1, rocss_tasmax_2, rocss_tasmax_3, ncol = 2)
dev.off()

pdf 
  2

### obs y bC

In [14]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_ndays_tasmax_0_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmax_bc_0$Data, 
                                    obs = ndtasmax_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_1_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmax_bc_1$Data, 
                                    obs = ndtasmax_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_2_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmax_bc_2$Data, 
                                    obs = ndtasmax_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmax_3_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmax_bc_3$Data, 
                                    obs = ndtasmax_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_tasmax_0_bc = makeMultiGrid(lapply(rocss_ndays_tasmax_0_bc[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_1_bc = makeMultiGrid(lapply(rocss_ndays_tasmax_1_bc[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_2_bc = makeMultiGrid(lapply(rocss_ndays_tasmax_2_bc[1:3], "easyVeri2grid", ndtasmax_obs))
mg_tasmax_3_bc = makeMultiGrid(lapply(rocss_ndays_tasmax_3_bc[1:3], "easyVeri2grid", ndtasmax_obs))

In [18]:
# Aplico función de bootstraping
rocss_bootstrap_tasmax_0_bc = rocss_bootstrap(ndtasmax_bc_0, ndtasmax_obs)
rocss_bootstrap_tasmax_1_bc = rocss_bootstrap(ndtasmax_bc_1, ndtasmax_obs)
rocss_bootstrap_tasmax_2_bc = rocss_bootstrap(ndtasmax_bc_2, ndtasmax_obs)
rocss_bootstrap_tasmax_3_bc = rocss_bootstrap(ndtasmax_bc_3, ndtasmax_obs)

In [ ]:
saveRDS(rocss_bootstrap_tasmax_0_bc, file = "rocss_bootstrap_tasmax_bc_oliv_0.rds")
saveRDS(rocss_bootstrap_tasmax_1_bc, file = "rocss_bootstrap_tasmax_bc_oliv_1.rds")
saveRDS(rocss_bootstrap_tasmax_2_bc, file = "rocss_bootstrap_tasmax_bc_oliv_2.rds")
saveRDS(rocss_bootstrap_tasmax_3_bc, file = "rocss_bootstrap_tasmax_bc_oliv_3.rds")

In [ ]:
rocss_bootstrap_tasmax_0 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_bc_oliv_0.rds")
rocss_bootstrap_tasmax_1 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_bc_oliv_1.rds")
rocss_bootstrap_tasmax_2 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_bc_oliv_2.rds")
rocss_bootstrap_tasmax_3 = readRDS("../../data/vine_data/rocss_bootstrap_tasmax_bc_oliv_3.rds")

In [22]:
# Aplico función de máscara significativa
pts_layers_tasmax_0_bc = get_rocss_stippling_layers(mg_tasmax_0_bc, rocss_bootstrap_tasmax_0_bc, ndtasmax_bc_0)
pts_layers_tasmax_1_bc = get_rocss_stippling_layers(mg_tasmax_1_bc, rocss_bootstrap_tasmax_1_bc, ndtasmax_bc_0)
pts_layers_tasmax_2_bc = get_rocss_stippling_layers(mg_tasmax_2_bc, rocss_bootstrap_tasmax_2_bc, ndtasmax_bc_0)
pts_layers_tasmax_3_bc = get_rocss_stippling_layers(mg_tasmax_3_bc, rocss_bootstrap_tasmax_3_bc, ndtasmax_bc_0)

In [23]:
rocss_tasmax_0_bc = spatialPlot(climatology(mg_tasmax_0_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color,at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmax_0_bc),
                                main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_1_bc = spatialPlot(climatology(mg_tasmax_1_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color,at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmax_1_bc),
                                main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_2_bc = spatialPlot(climatology(mg_tasmax_2_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color,at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmax_2_bc),
                                main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss_tasmax_3_bc = spatialPlot(climatology(mg_tasmax_3_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color,at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmax_3_bc),
                                main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [24]:
png("rocss_ndays_tmax_bc_oliv.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_tasmax_0_bc, rocss_tasmax_1_bc, rocss_tasmax_2_bc, rocss_tasmax_3_bc, ncol = 2)
dev.off()

pdf 
  2

## ROCSS ndays tmin

### obs y raw

In [12]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_ndays_tasmin_0 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmin_0$Data, 
                                 obs = ndtasmin_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_1 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmin_1$Data, 
                                 obs = ndtasmin_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_2 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmin_2$Data, 
                                 obs = ndtasmin_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_3 = veriApply(verifun = "EnsRocss", 
                                 fcst = ndtasmin_3$Data, 
                                 obs = ndtasmin_obs$Data,
                                 prob = c(1/3, 2/3),
                                 ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_tasmin_0 = makeMultiGrid(lapply(rocss_ndays_tasmin_0[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_1 = makeMultiGrid(lapply(rocss_ndays_tasmin_1[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_2 = makeMultiGrid(lapply(rocss_ndays_tasmin_2[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_3 = makeMultiGrid(lapply(rocss_ndays_tasmin_3[1:3], "easyVeri2grid", ndtasmin_obs))

In [13]:
# Aplico función de bootstraping
rocss_bootstrap_tasmin_0 = rocss_bootstrap(ndtasmin_0, ndtasmin_obs)
rocss_bootstrap_tasmin_1 = rocss_bootstrap(ndtasmin_1, ndtasmin_obs)
rocss_bootstrap_tasmin_2 = rocss_bootstrap(ndtasmin_2, ndtasmin_obs)
rocss_bootstrap_tasmin_3 = rocss_bootstrap(ndtasmin_3, ndtasmin_obs)

In [ ]:
saveRDS(rocss_bootstrap_tasmin_0, file = "rocss_bootstrap_tasmin_raw_oliv_0.rds")
saveRDS(rocss_bootstrap_tasmin_1, file = "rocss_bootstrap_tasmin_raw_oliv_1.rds")
saveRDS(rocss_bootstrap_tasmin_2, file = "rocss_bootstrap_tasmin_raw_oliv_2.rds")
saveRDS(rocss_bootstrap_tasmin_3, file = "rocss_bootstrap_tasmin_raw_oliv_3.rds")

In [ ]:
rocss_bootstrap_tasmin_0 = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_raw_oliv_0.rds")
rocss_bootstrap_tasmin_1 = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_raw_oliv_1.rds")
rocss_bootstrap_tasmin_2 = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_raw_oliv_2.rds")
rocss_bootstrap_tasmin_3 = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_raw_oliv_3.rds")

In [16]:
# Aplico función de máscara significativa
pts_layers_tasmin_0 = get_rocss_stippling_layers(mg_tasmin_0, rocss_bootstrap_tasmin_0, ndtasmin_0)
pts_layers_tasmin_1 = get_rocss_stippling_layers(mg_tasmin_1, rocss_bootstrap_tasmin_1, ndtasmin_0)
pts_layers_tasmin_2 = get_rocss_stippling_layers(mg_tasmin_2, rocss_bootstrap_tasmin_2, ndtasmin_0)
pts_layers_tasmin_3 = get_rocss_stippling_layers(mg_tasmin_3, rocss_bootstrap_tasmin_3, ndtasmin_0)

In [17]:
rocss_tasmin_0 = spatialPlot(climatology(mg_tasmin_0),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmin_0),
                             main = "Raw (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_1 = spatialPlot(climatology(mg_tasmin_1),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmin_1),
                             main = "Raw (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_2 = spatialPlot(climatology(mg_tasmin_2),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmin_2),
                             main = "Raw (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_3 = spatialPlot(climatology(mg_tasmin_3),
                             backdrop.theme = "countries",
                             names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                             layout = c(3,1),
                             col.regions = color,
                             at = seq(-1, 1, 0.05),
                             sp.layout = list(pts_layers_tasmin_3),
                             main = "Raw (lt 3)") %>% suppressMessages %>% suppressWarnings

In [18]:
png("rocss_ndays_tmin_raw_oliv.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_tasmin_0, rocss_tasmin_1, rocss_tasmin_2, rocss_tasmin_3, ncol = 2)
dev.off()

pdf 
  2

### obs y bC

In [12]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_ndays_tasmin_0_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmin_bc_0$Data, 
                                    obs = ndtasmin_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_1_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmin_bc_1$Data, 
                                    obs = ndtasmin_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_2_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmin_bc_2$Data, 
                                    obs = ndtasmin_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

rocss_ndays_tasmin_3_bc = veriApply(verifun = "EnsRocss", 
                                    fcst = ndtasmin_bc_3$Data, 
                                    obs = ndtasmin_obs$Data,
                                    prob = c(1/3, 2/3),
                                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_tasmin_0_bc = makeMultiGrid(lapply(rocss_ndays_tasmin_0_bc[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_1_bc = makeMultiGrid(lapply(rocss_ndays_tasmin_1_bc[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_2_bc = makeMultiGrid(lapply(rocss_ndays_tasmin_2_bc[1:3], "easyVeri2grid", ndtasmin_obs))
mg_tasmin_3_bc = makeMultiGrid(lapply(rocss_ndays_tasmin_3_bc[1:3], "easyVeri2grid", ndtasmin_obs))

In [13]:
# Aplico función de bootstraping
rocss_bootstrap_tasmin_0_bc = rocss_bootstrap(ndtasmin_bc_0, ndtasmin_obs)
rocss_bootstrap_tasmin_1_bc = rocss_bootstrap(ndtasmin_bc_1, ndtasmin_obs)
rocss_bootstrap_tasmin_2_bc = rocss_bootstrap(ndtasmin_bc_2, ndtasmin_obs)
rocss_bootstrap_tasmin_3_bc = rocss_bootstrap(ndtasmin_bc_3, ndtasmin_obs)

In [ ]:
saveRDS(rocss_bootstrap_tasmin_0_bc, file = "rocss_bootstrap_tasmin_bc_oliv_0.rds")
saveRDS(rocss_bootstrap_tasmin_1_bc, file = "rocss_bootstrap_tasmin_bc_oliv_1.rds")
saveRDS(rocss_bootstrap_tasmin_2_bc, file = "rocss_bootstrap_tasmin_bc_oliv_2.rds")
saveRDS(rocss_bootstrap_tasmin_3_bc, file = "rocss_bootstrap_tasmin_bc_oliv_3.rds")

In [ ]:
rocss_bootstrap_tasmin_0_bc = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_bc_oliv_0.rds")
rocss_bootstrap_tasmin_1_bc = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_bc_oliv_1.rds")
rocss_bootstrap_tasmin_2_bc = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_bc_oliv_2.rds")
rocss_bootstrap_tasmin_3_bc = readRDS("../../data/vine_data/rocss_bootstrap_tasmin_bc_oliv_3.rds")

In [16]:
# Aplico función de máscara significativa
pts_layers_tasmin_0_bc = get_rocss_stippling_layers(mg_tasmin_0_bc, rocss_bootstrap_tasmin_0_bc, ndtasmin_bc_0)
pts_layers_tasmin_1_bc = get_rocss_stippling_layers(mg_tasmin_1_bc, rocss_bootstrap_tasmin_1_bc, ndtasmin_bc_0)
pts_layers_tasmin_2_bc = get_rocss_stippling_layers(mg_tasmin_2_bc, rocss_bootstrap_tasmin_2_bc, ndtasmin_bc_0)
pts_layers_tasmin_3_bc = get_rocss_stippling_layers(mg_tasmin_3_bc, rocss_bootstrap_tasmin_3_bc, ndtasmin_bc_0)

In [17]:
rocss_tasmin_0_bc = spatialPlot(climatology(mg_tasmin_0_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color, at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmin_0_bc),
                                main = "BC (lt 0)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_1_bc = spatialPlot(climatology(mg_tasmin_1_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color, at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmin_1_bc),
                                main = "BC (lt 1)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_2_bc = spatialPlot(climatology(mg_tasmin_2_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color, at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmin_2_bc),
                                main = "BC (lt 2)") %>% suppressMessages %>% suppressWarnings

rocss_tasmin_3_bc = spatialPlot(climatology(mg_tasmin_3_bc),
                                backdrop.theme = "countries",
                                names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                                layout = c(3,1),
                                col.regions = color, at = seq(-1, 1, 0.05),
                                sp.layout = list(pts_layers_tasmin_3_bc),
                                main = "BC (lt 3)") %>% suppressMessages %>% suppressWarnings

In [18]:
png("rocss_ndays_tmin_bc_oliv.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_tasmin_0_bc, rocss_tasmin_1_bc, rocss_tasmin_2_bc, rocss_tasmin_3_bc, ncol = 2)
dev.off()

pdf 
  2